In [1]:
import boto3

# 載入 .env 檔案中的環境變數
from dotenv import load_dotenv
import os
load_dotenv()

# 建立 STS 客戶端
sts_client = boto3.client("sts")

# 呼叫 get_caller_identity 以取得當前 AWS 帳號資訊
response = sts_client.get_caller_identity()
account_id = response["Account"]

print(f"AWS Account ID: {account_id}")

AWS Account ID: 339712730744


In [2]:
# 建立 IAM 客戶端
iam_client = boto3.client("iam")

# 取得所有角色
def list_roles():
    try:
        # 透過客戶端取得所有角色
        response = iam_client.list_roles()

        # 儲存所有角色的名稱和 ARN
        roles = {role["RoleName"]: role["Arn"] for role in response["Roles"]}
        return roles
    except Exception as e:
        print(f"無法取得角色列表。錯誤訊息：{e}")
        return None


# 自訂義篩選角色的函數，函數需傳入關鍵字進行篩選
def get_role_arn(role_keyword):
    try:
        # 呼叫 list_roles 方法以取得所有角色
        response = iam_client.list_roles()
        # 遍歷所有角色以篩選出包含指定關鍵字的角色 ARN
        for role in response["Roles"]:
            if role_keyword in role["RoleName"]:
                return role["Arn"]
        print(f"無法找到包含關鍵字 '{role_keyword}' 的角色。")
        return None
    except Exception as e:
        print(f"無法取得角色列表。錯誤訊息：{e}")
        return None


# 定義要搜尋的角色關鍵字
database_role_keyword = "ComprehendDataAccessRole"
translate_role_keyword = "TranslateDemoRole"

# 取得包含 'ComprehendDataAccessRole' 的角色 ARN
database_access_role_arn = get_role_arn(
    database_role_keyword
)
# 取得包含 'TranslateDemoRole' 的角色 ARN
translate_access_role_arn = get_role_arn(
    translate_role_keyword
)

# 輸出查看
print(
    f"Database Access Role ARN：【{database_access_role_arn}】"
)
print(
    f"Translate Access Role ARN：【{translate_access_role_arn}】"
)

Database Access Role ARN：【arn:aws:iam::339712730744:role/service-role/c133864a3391506l8170626t1w-ComprehendDataAccessRole-4KfsM2OyUHE4】
Translate Access Role ARN：【arn:aws:iam::339712730744:role/c133864a3391506l8170626t1w3397127-TranslateDemoRole-YJ246eTks6oS】


In [3]:
# 建立 S3 客戶端
s3_client = boto3.client("s3")

# 列出 Buckets
def list_buckets():
    try:
        # 呼叫 list_buckets 方法以取得所有 bucket
        response = s3_client.list_buckets()

        # 從回應中提取每個 bucket 名稱並篩選包含 'labbucket' 的
        bucket_names = [
            bucket["Name"]
            for bucket in response["Buckets"]
            if "labbucket" in bucket["Name"]
        ]
        return bucket_names
    except Exception as e:
        print(f"無法取得 bucket 列表。錯誤訊息：{e}")
        return None


# 取得篩選後的 bucket 名稱
bucket_names = list_buckets()

# 顯示取得的 bucket 名稱
if bucket_names:
    print("包含 'labbucket' 的 Bucket 名稱列表：")
    for name in bucket_names:
        print(name)
else:
    print("沒有找到包含 'labbucket' 的 Bucket。")

包含 'labbucket' 的 Bucket 名稱列表：
c133864a3391506l8170626t1w339712730744-labbucket-ewaj36onfsdc


In [4]:
# 生成全域唯一識別碼
import uuid

# 處理 JSON 格式的數據
import json

# 從 time 模組匯入 sleep 函數，用於讓程式暫停指定的秒數
from time import sleep

# 自訂義篩選 Bucket 的函數，需傳入關鍵字進行篩選
def list_buckets_with_keyword(keyword):
    try:
        # 調用 list_buckets 取得所有 bucket
        response = s3_client.list_buckets()
        bucket_names = [
            # 篩選包含特定關鍵字的名稱
            # 特別注意這是 inline 的表達式，斷行不要縮排
            bucket["Name"]
            for bucket in response["Buckets"]
            if keyword in bucket["Name"]
        ]
        if bucket_names:
            # 假設只取第一個符合條件的 bucket 名稱
            return bucket_names[0]
        else:
            print(f"沒有找到包含 '{keyword}' 的 bucket。")
            return None
    except Exception as e:
        print(f"無法取得 bucket 列表。錯誤訊息：{e}")
        return None


# 取得包含 'labbucket' 的 S3 bucket 名稱
bucket = list_buckets_with_keyword("labbucket")

# 輸出以上步驟取得的 ARN
print(f"S3 Bucket 名稱：【{bucket}】")

S3 Bucket 名稱：【c133864a3391506l8170626t1w339712730744-labbucket-ewaj36onfsdc】


In [5]:
media_input_uri = f"s3://{bucket}/lab71/transcribe-sample/test.wav"

In [6]:
# 建立 AWS Transcribe 的客戶端
transcribe_client = boto3.client("transcribe")

# 生成 UUID 作為此轉錄作業的名稱
job_uuid = uuid.uuid1()
# 組合轉錄作業名稱
transcribe_job_name = f"transcribe-job-{job_uuid}"
# 自訂義轉錄輸出的檔案名稱
transcribe_output_filename = "transcribe_output.txt"

# 啟動轉錄作業
response = transcribe_client.start_transcription_job(
    # 轉錄作業的名稱
    TranscriptionJobName=transcribe_job_name,
    # 轉錄的媒體檔案 URI
    Media={"MediaFileUri": media_input_uri},
    MediaFormat="wav",
    # 音訊中的語言代碼
    LanguageCode="en-US",
    # 轉錄結果的輸出 S3 bucket
    OutputBucketName=bucket,
    # 輸出結果檔案的名稱
    OutputKey=transcribe_output_filename,
)

In [7]:
response

{'TranscriptionJob': {'TranscriptionJobName': 'transcribe-job-51977ffc-967c-11ef-a60a-3a328b8844c7',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'en-US',
  'MediaFormat': 'wav',
  'Media': {'MediaFileUri': 's3://c133864a3391506l8170626t1w339712730744-labbucket-ewaj36onfsdc/lab71/transcribe-sample/test.wav'},
  'StartTime': datetime.datetime(2024, 10, 30, 13, 3, 36, 733000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 10, 30, 13, 3, 36, 712000, tzinfo=tzlocal())},
 'ResponseMetadata': {'RequestId': '28fd52bb-6a2e-4b90-84bb-5c01a5aca130',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '28fd52bb-6a2e-4b90-84bb-5c01a5aca130',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '369',
   'date': 'Wed, 30 Oct 2024 05:03:36 GMT'},
  'RetryAttempts': 0}}